## Environment preparation

In [ ]:
# Load `.env` variables. If nothing provided, synthetic data will be used instead.
import dotenv
dotenv.load_dotenv(override=True)

In [ ]:
# Not necessary, but useful for live-reloading changes to Obsinthe itself.
%load_ext autoreload
%autoreload 2

In [ ]:
# Some core dependencies.
import os
from datetime import timedelta, datetime

# For printing values to notebook.
from IPython.display import display, HTML

# For talking to Prometheus
from obsinthe.prometheus.client import Client
from obsinthe.prometheus.loader import Loader

# For simulating alerts data (when not connected to live source)
from obsinthe.testing.prometheus.client import MockedClient
from obsinthe.testing.prometheus.alerts import AlertsDatasetBuilder

# For merging daily data into a single dataset
from obsinthe.prometheus.data import intervals_concat_days

# For alerts clusterin
from obsinthe.alerts.grouping import alerts_groups_one_hot, alerts_clustering_dbscan

# For visualization
from obsinthe.vis.alerts import plot_alerts_timeline
from obsinthe.vis.clustering import plot_clustering

# To avoid some issues when rendering Plotly on export to HTML
import cufflinks
cufflinks.go_offline()

## Data loading

In [ ]:
# What date range we want to load the data for.
START = datetime(2024, 3, 19, 10, 10)
END = datetime(2024, 3, 23, 0, 0)

In [ ]:
# Load the data or simulate them when endpoint not available.

# Replace with your instance, e.g. "https://prometheus.example.com".
PROM_URL = ""

if not PROM_URL:
    client = MockedClient(AlertsDatasetBuilder(START, END))
    client.mock_setup((START, END))
else:
    # We expect the token being available in `PROM_TOKEN`. Add it to `.env` file
    client = Client(url=PROM_URL, token=os.getenv("PROM_TOKEN"))

# Load alerts data from the Prometheus instance.
loader = Loader(client)

alerts_ranges_collection = loader.interval_query("ALERTS[24h:1m]", START, END)
alerts_ranges_collection[0].df

## Data transformation

In [ ]:
alerts_intervals_collection = alerts_ranges_collection.fmap(
    lambda ds: ds.to_intervals_ds(timedelta(minutes=1))
)
alerts_intervals_collection[0].df

In [ ]:
alerts_intervals = intervals_concat_days(
    alerts_intervals_collection
).correct_for_resolution(timedelta(minutes=1))
alerts_intervals.df

## Identifying groups of alerts

In [ ]:
alert_id = lambda a: f"{a['alertname']}-{a['instance_id']}"

In [ ]:
plot_alerts_timeline(alerts_intervals, alert_id=alert_id).show()

plot_alerts_timeline(
    alerts_intervals.fmap(lambda df: df.query("instance_id == '1'")), alert_id=alert_id
).show()

In [ ]:
# Group alert starting within provided tolerange and turn data into one-hot encoding.
one_hot = alerts_groups_one_hot(
    alerts_intervals,
    groupby_columns=["instance_id"],
    group_tolerance=timedelta(minutes=3),
)

one_hot

## Apply clustering alogorithm

In [ ]:
# Apply the clustering.
ac = alerts_clustering_dbscan(
    one_hot,
    eps=1,
    n_neighbors=2,
    min_samples=2,
    min_dist=0.1,
)

plot_clustering(ac)

## Additional data cleanup

In [ ]:
# Show case with flapping alerts.

fig_flap = plot_alerts_timeline(
    alerts_intervals.fmap(
        lambda df: df.query("instance_id == '10'")
    ),
    alert_id=alert_id,
    height=600
)

display(HTML("<h3>With flapping</h3>"))
fig_flap.show()

In [ ]:
# Reduce the flapping by merging with positive threshold.
alerts_intervals_reduced_flap = alerts_intervals.merge_overlaps(
    threshold=timedelta(minutes=30)
)

fig_noflap = plot_alerts_timeline(
    alerts_intervals_reduced_flap.fmap(
        lambda df: df.query("instance_id == '10'")
    ),
    alert_id=alert_id,
    height=600
)

display(HTML("<h3>Without flapping</h3>"))
fig_noflap.show()

## Re-apply the clustering after the cleanup

In [ ]:
# To compare with previous version.
one_hot_noflap = alerts_groups_one_hot(
    alerts_intervals_reduced_flap,
    groupby_columns=["instance_id"],
    group_tolerance=timedelta(minutes=3),
)

# Apply clustering after flapping reduction.
ac_noflap = alerts_clustering_dbscan(
    one_hot_noflap,
    eps=1,
    n_neighbors=2,
    min_samples=2,
    min_dist=0.1,
)

display(HTML("<h3>With flapping</h3>"))
plot_clustering(ac).show()

display(HTML("<h3>Without flapping</h3>"))
plot_clustering(ac_noflap).show()

## Examing relative risks between alerts combinations

In [ ]:
from obsinthe.utils.relative_risks import RelativeRisks

In [ ]:
rr_calc = RelativeRisks(one_hot_noflap, one_hot_noflap)
rr_calc.calculate()
rr_calc.where(rr_calc.E_and_O > 5)